<a href="https://colab.research.google.com/github/cpptake/CommonLit/blob/main/roberta_large_mean_pooling_model_cossche_layernorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mean Pooling model LB:0.468

元Note：https://www.kaggle.com/jcesquiveld/roberta-large-5-fold-single-model-meanpooling
<br>https://www.kaggle.com/jcesquiveld/best-transformer-representations

# prerequisite


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jul 23 02:43:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# バージョン更新
!pip uninstall -r /content/drive/MyDrive/CommonLit/working/requirements.txt -y


Found existing installation: pandas 1.1.5
Uninstalling pandas-1.1.5:
  Successfully uninstalled pandas-1.1.5
Found existing installation: sklearn 0.0
Uninstalling sklearn-0.0:
  Successfully uninstalled sklearn-0.0
Found existing installation: sklearn-pandas 1.8.0
Uninstalling sklearn-pandas-1.8.0:
  Successfully uninstalled sklearn-pandas-1.8.0
Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102


In [4]:
!pip install -r /content/drive/MyDrive/CommonLit/working/requirements.txt


     |████████████████████████████████| 9.9 MB 7.1 MB/s 
     |████████████████████████████████| 776.7 MB 4.4 kB/s 
     |████████████████████████████████| 176 kB 73.6 MB/s 
     |████████████████████████████████| 12.7 MB 9.2 MB/s 
     |████████████████████████████████| 2.1 MB 55.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 28.5 MB 54 kB/s 
     |████████████████████████████████| 3.3 MB 19.3 MB/s 
     |████████████████████████████████| 895 kB 63.0 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=16f4b2518743b8977166b836b666923c345fb3a203fcdd483eb89fc2cbce7811
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: scikit-learn
    Fou

In [5]:
!pip freeze |grep -e random -e math -e numpy -e pandas -e torch -e transformers -e sklearn -e gc

mpmath==1.2.1
numpy==1.19.5
pandas==1.2.3
pandas-datareader==0.9.0
pandas-gbq==0.13.3
pandas-profiling==1.4.1
sklearn==0.0
sklearn-pandas==2.1.0
tensorflow-gcs-config==2.5.0
torch==1.7.0
torchmetrics==0.2.0
torchsummary==1.5.1
torchtext==0.10.0
torchvision==0.8.1
transformers==4.5.1


## kaggle.json認証

In [6]:
# !mkdir -p /root/.kaggle/
# !cp ./drive/MyDrive/kaggle/commonlit/kaggle.json ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json

!pip install -q kaggle
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json /root/.kaggle/

# !ls ../input/commonlitreadabilityprize/
!ls /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize

clrp-roberta-base		    roberta-base-nishipy
clrprobertalarge		    roberta-large
clr-roberta			    roberta-large-itptfit
commonlit-readability-roberta-base  roberta-large-mlm
dataset-metadata.json		    sample_submission.csv
huggingface-roberta-variants	    test.csv
__MACOSX			    train.csv
pretrained-model


In [7]:
# !mkdir -p /content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize/pretrained-model/
# !kaggle datasets download iamnishipy/roberta-large-20210712191259-mlm

In [8]:
!unzip -o ./roberta-large-20210712191259-mlm.zip -d ../input/commonlitreadabilityprize/pretrained-model/

unzip:  cannot find or open ./roberta-large-20210712191259-mlm.zip, ./roberta-large-20210712191259-mlm.zip.zip or ./roberta-large-20210712191259-mlm.zip.ZIP.


In [9]:
!ls ../input/commonlitreadabilityprize/pretrained-model/

ls: cannot access '../input/commonlitreadabilityprize/pretrained-model/': No such file or directory


# Import

In [10]:
!pip install transformers accelerate datasets

     |████████████████████████████████| 49 kB 3.9 MB/s 
     |████████████████████████████████| 542 kB 15.8 MB/s 
     |████████████████████████████████| 118 kB 60.2 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 243 kB 29.4 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.


In [11]:
# from tqdm.contrib.concurrent import process_map#, thread_map
# r = process_map(myfunc, args, max_workers=16, chunksize=100)

In [12]:
# Imports

import os
import numpy as np
import pandas as pd
import random

from transformers import AutoConfig, AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, logging,get_cosine_schedule_with_warmup
logging.set_verbosity_error()

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, SequentialSampler, RandomSampler, DataLoader

from tqdm.notebook import tqdm

import gc; gc.enable()
from IPython.display import clear_output

from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')


In [13]:
# Definitions

INPUT_DIR = '/content/drive/MyDrive/CommonLit/input/commonlitreadabilityprize'
MODEL_NAME = 'roberta-large'

HIDDEN_SIZE = 1024
NUM_HIDDEN_LAYERS = 24
LAYER_START = 4   # for WeightedLayerPoolingModel

HIDDEN_DIM_FC = 128    # for AttentionPooling

MAX_LENGTH = 300
LR = 2e-5
EPS = 1e-8

SEED = 42

NUM_FOLDS = 5
SEEDS = [113, 71, 17, 43, 37]

EPOCHS = 5
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 32

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# Utilities

# Utility classes and functions

class ContinuousStratifiedKFold(StratifiedKFold):
    def split(selfself, x, y, groups=None):
        num_bins = int(np.floor(1 + np.log2(len(y))))
        bins = pd.cut(y, bins=num_bins, labels=False)
        return super().split(x, bins, groups)
    
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_data_loaders(data, fold):
    
    x_train = data.loc[data.fold != fold, 'excerpt'].tolist()
    y_train = data.loc[data.fold != fold, 'target'].values
    x_val = data.loc[data.fold == fold, 'excerpt'].tolist()
    y_val = data.loc[data.fold == fold, 'target'].values
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    encoded_train = tokenizer.batch_encode_plus(
        x_train, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )
    
    encoded_val = tokenizer.batch_encode_plus(
        x_val, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )
    
    dataset_train = TensorDataset(
        encoded_train['input_ids'],
        encoded_train['attention_mask'],
        torch.tensor(y_train)
    )
    dataset_val = TensorDataset(
        encoded_val['input_ids'],
        encoded_val['attention_mask'],
        torch.tensor(y_val)
    )
    
    dataloader_train = DataLoader(
        dataset_train,
        sampler = RandomSampler(dataset_train),
        batch_size=TRAIN_BATCH_SIZE
    )

    dataloader_val = DataLoader(
        dataset_val,
        sampler = SequentialSampler(dataset_val),
        batch_size=VAL_BATCH_SIZE
    )

    return dataloader_train, dataloader_val

<h2>Model definitions</h2>

In [15]:
# Model definitions

class PoolerModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        config.update({'num_labels':1})
        self.model = AutoModel.from_pretrained(model_name, config)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        pooler_output = outputs[1]
        logits = self.linear(pooler_output)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
        
class LastLayerCLSModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        last_hidden_state = outputs[0]
        cls_embeddings = last_hidden_state[:,0]
        logits = self.linear(cls_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
        
class MeanPoolingModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        ####追加
        self.layer_norm = nn.LayerNorm(1024)
        ####
        logits = self.linear(mean_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
        
class MaxPoolingModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        last_hidden_state[input_mask_expanded == 0] = -1e9   # large negative value
        max_embeddings, _ = torch.max(last_hidden_state, 1)
        logits = self.linear(max_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
        
class SecondToLastLayerCLSModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        config.update({'output_hidden_states':True})
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        all_hidden_states = torch.stack(outputs[2])
        second_to_last_layer = 23
        cls_embeddings = all_hidden_states[second_to_last_layer,:,0]
        logits = self.linear(cls_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
class ConcatenateLastFourModel(nn.Module):
    
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        config.update({'output_hidden_states':True})
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.linear = nn.Linear(4*HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        all_hidden_states = torch.stack(outputs[2])
        concatenate_pooling = torch.cat(
            (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]), -1
        )
        concatenate_pooling = concatenate_pooling[:,0]
        logits = self.linear(concatenate_pooling)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
        
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average
    
class WeightedLayerPoolingModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        config.update({'output_hidden_states':True})
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.pooling = WeightedLayerPooling(NUM_HIDDEN_LAYERS, 
                                      layer_start=LAYER_START,
                                      layer_weights=None)
        self.layer_norm = nn.LayerNorm(HIDDEN_SIZE)
        self.linear = nn.Linear(HIDDEN_SIZE, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        all_hidden_states = torch.stack(outputs[2])
        
        weighted_pooling_embeddings = self.pooling(all_hidden_states)
        weighted_pooling_embeddings = weighted_pooling_embeddings[:,0]
        
        norm_embeddings = self.layer_norm(weighted_pooling_embeddings)
        logits = self.linear(norm_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        
class AttentionPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_fc):
        super(AttentionPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_fc = hiddendim_fc
        self.dropout = nn.Dropout(0.1)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.hidden_size))
        self.q = nn.Parameter(torch.from_numpy(q_t)).float().to(DEVICE)
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.hidden_size, self.hiddendim_fc))
        self.w_h = nn.Parameter(torch.from_numpy(w_ht)).float().to(DEVICE)

    def forward(self, all_hidden_states):
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers+1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out = self.attention(hidden_states)
        out = self.dropout(out)
        return out

    def attention(self, h):
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = F.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        return v

class AttentionPoolingModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_name)
        config.update({'output_hidden_states':True})
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.pooler = AttentionPooling(NUM_HIDDEN_LAYERS, HIDDEN_SIZE, HIDDEN_DIM_FC)
        
        self.linear = nn.Linear(HIDDEN_DIM_FC, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        
        outputs = self.model(input_ids, attention_mask)
        all_hidden_states = torch.stack(outputs[2])
        
        attention_pooling_embeddings = self.pooler(all_hidden_states)
        
        logits = self.linear(attention_pooling_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = self.loss(preds.view(-1).float(), labels.view(-1).float())
            return loss
        else:
            return preds
        

# 評価関数

In [16]:
def eval_mse(model, data_loader):
    """Evaluates the mean squared error of the |model| on |data_loader|"""
    model.eval()            
    mse_sum = 0

    with torch.no_grad():
        for batch_num, (input_ids, attention_mask, target) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)                        
            target = target.to(DEVICE)           
            
            pred = model(input_ids, attention_mask)                       

            mse_sum += nn.MSELoss(reduction="sum")(pred.flatten(), target).item()
                

    return mse_sum / len(data_loader.dataset)

<h2>Read data and create folds</h2>

For the results to be reproducible, I create the same folds I'm using in the competition.

In [17]:
# Read competition data
data = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))

# Create stratified folds
kf = ContinuousStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
for f, (t_, v_) in enumerate(kf.split(data, data.target)):
    data.loc[v_, 'fold'] = f
data['fold'] = data['fold'].astype(int)


<h2>Training and evaluation</h2>

In [18]:
!mkdir /content/drive/MyDrive/CommonLit/output/mean-pooling-model-cossche-layernorm

In [19]:
def evaluate(model, val_dataloader):

    model.eval()
    
    loss_val_total = 0
    
    for batch in val_dataloader:
        
        batch = tuple(b.to(DEVICE) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            loss = model(**inputs)
            
        loss_val_total += loss.item()

    loss_val_avg = loss_val_total/len(val_dataloader) 
            
    return loss_val_avg


def train(model, train_dataloader, val_dataloader):
    
    optimizer = AdamW(model.parameters(), lr = LR, eps = EPS)

    print("len(train_dataloader) = ",len(train_dataloader))
    
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, 
                                                num_training_steps=len(train_dataloader) * EPOCHS)
    # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, 
    #                                             num_training_steps=len(train_dataloader) * EPOCHS)
    best_val_loss = 1
    best_rmse = 10
    model.train()
                                                
    for epoch in range(EPOCHS):
    
        loss_train_total = 0
        for batch in tqdm(train_dataloader):
    
            model.zero_grad()
            batch = tuple(b.to(DEVICE) for b in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }
        
            loss = model(**inputs)
            loss_train_total += loss.item()
        
            loss.backward()
            optimizer.step()
            scheduler.step()
        
        loss_train_avg = loss_train_total / len(train_dataloader)
        
        val_rmse = math.sqrt(eval_mse(model, val_dataloader))
        loss_val_avg = evaluate(model, val_dataloader)
        print(f'epoch:{epoch+1}/{EPOCHS} train loss={loss_train_avg}  val loss={loss_val_avg}  val rmse = {val_rmse}')

        if val_rmse < best_rmse:
            print("best rmse is Updated! ",best_rmse , " to ", val_rmse)
            best_rmse = val_rmse
            torch.save(model.state_dict(),f"/content/drive/MyDrive/CommonLit/output/mean-pooling-model-cossche-layernorm/model_fold{fold}.pth")
   
        # if loss_val_avg < best_val_loss:
        #     print("best val loss is Updated! ",best_val_loss , " to ", loss_val_avg)
        #     best_val_loss = loss_val_avg
        #     torch.save(model.state_dict(),f"model_fold{fold}.pth")
            
    return best_rmse

In [ ]:
#  if loss_val_avg < best_val_loss:
#     best_val_loss = loss_val_avg
#     torch.save(model.state_dict(),f"model{fold}.bin")

The process takes more than eight hours with Kaggle kernels, so I've included a termination condition for MAX_RUNS=2 runs (instead of the 5 seeds x 5 folds = 25 runs). Just remove it in both loops to run the full experiment.

In [20]:
# Main loop
import math

losses = []

MAX_RUNS = 1
runs = 0   # Variable to control termination condition


for i, seed in enumerate(SEEDS):
       
    # Termination condition
    if runs == MAX_RUNS:
        print(f'{runs} runs termination condition reached.')
        break    
    
    print(f'********* seed({i}) = {seed} ***********')
    
    for fold in range(NUM_FOLDS):
        print(f'*** fold = {fold} ***')
        seed_everything(seed)
        train_dataloader, val_dataloader = get_data_loaders(data, fold)
        
        # Change the instantiated model by LastLayerCLSModel, PoolingModel, etc. 
        model = MeanPoolingModel(MODEL_NAME)
        model.to(DEVICE)
        
        loss = train(model, train_dataloader, val_dataloader)
        losses.append(loss)

        print('RMSE mean = ',np.array(losses).mean())
        print('RMSE each fold = ',losses)
        print('  ')
        
        del model
        gc.collect()
        
    # Termination condition
    runs += 1
    if runs == MAX_RUNS:
        break
    

********* seed(0) = 113 ***********
*** fold = 0 ***


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

epoch:3/5 train loss=0.05428795166097214  val loss=0.2555692005488608  val rmse = 0.5057743408104953


  0%|          | 0/284 [00:00<?, ?it/s]

epoch:4/5 train loss=0.014904320248219013  val loss=0.2378629239069091  val rmse = 0.48884720738175713
best rmse is Updated!  0.502174748473946  to  0.48884720738175713


  0%|          | 0/284 [00:00<?, ?it/s]

epoch:5/5 train loss=0.005463089992933329  val loss=0.23848877847194672  val rmse = 0.48940000297011754
RMSE mean =  0.4812578350859072
RMSE each fold =  [0.4730299878934801, 0.5006104422461594, 0.48902675647990496, 0.45477478142823424, 0.48884720738175713]
  


<h2>Results</h2>

In [ ]:
# Save the losses (a column per model)

# Read the results of the previous models, if any
try:
    results = pd.read_csv('experiment-1.csv')
except:
    results = pd.DataFrame(data=None)
    
# Change the column with a name representative of the model: last_layer_cls, mean_pooling, etc.
results['pooler'] = losses
results.to_csv('experiment-1.csv', index=False)

In [ ]:
# Plot 95% confidence interval of the mean of the RMSE

fig, ax = plt.subplots(1,1,figsize=(15,8))
ax.set_xticks(np.arange(0.45, 0.54, step=0.005))
ax.set_axis_on()
sns.pointplot(data=np.sqrt(results), ci=95, orient='h', ax=ax)